In [7]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [11]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e8_path = kagglehub.competition_download('playground-series-s5e8')

print('Data source import complete.')
print(playground_series_s5e8_path)


Data source import complete.
/root/.cache/kagglehub/competitions/playground-series-s5e8


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np

In [12]:
train = pd.read_csv('/root/.cache/kagglehub/competitions/playground-series-s5e8/train.csv')
test = pd.read_csv('/root/.cache/kagglehub/competitions/playground-series-s5e8/test.csv')

In [14]:
train

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,services,single,secondary,no,1282,no,yes,unknown,4,jul,1006,2,-1,0,unknown,1
749996,749996,69,retired,divorced,tertiary,no,631,no,no,cellular,19,aug,87,1,-1,0,unknown,0
749997,749997,50,blue-collar,married,secondary,no,217,yes,no,cellular,17,apr,113,1,-1,0,unknown,0
749998,749998,32,technician,married,secondary,no,-274,no,no,cellular,26,aug,108,6,-1,0,unknown,0


In [15]:
# encoding categorical data
le = LabelEncoder()
train_en = train.copy()
for col in train_en[['job','marital','education','default','housing','loan','contact','month','poutcome']]:
    train_en[col]= le.fit_transform(train_en[col])
train_en

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,9,1,1,0,7,0,0,0,25,1,117,3,-1,0,3,0
1,1,38,1,1,1,0,514,0,0,2,18,6,185,1,-1,0,3,0
2,2,36,1,1,1,0,602,1,0,2,14,8,111,2,-1,0,3,0
3,3,27,8,2,1,0,34,1,0,2,28,8,10,2,-1,0,3,0
4,4,26,9,1,1,0,889,1,0,0,3,3,902,1,-1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,7,2,1,0,1282,0,1,2,4,5,1006,2,-1,0,3,1
749996,749996,69,5,0,2,0,631,0,0,0,19,1,87,1,-1,0,3,0
749997,749997,50,1,1,1,0,217,1,0,0,17,0,113,1,-1,0,3,0
749998,749998,32,9,1,1,0,-274,0,0,0,26,1,108,6,-1,0,3,0


In [16]:
#test validation split
from sklearn.model_selection import train_test_split
x_train = train_en.drop(columns = ['id','y'])
y_train = train_en['y']
xt,xv,yt,yv = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [17]:
model = LogisticRegression()
model.fit(xt, yt)
prob = model.predict(xv)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
prob

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
from sklearn.metrics import mean_squared_log_error
# --- RMSLE function
def rmsle(y_true, y_pred):
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [20]:
print(rmsle(yv,prob))

0.2227293765920892


In [21]:
test_en = test.copy()
test_en = test_en.drop(columns = ['id'])
for col in test_en[['job','marital','education','default','housing','loan','contact','month','poutcome']]:
    test_en[col]= le.fit_transform(test_en[col])

In [22]:
result = model.predict_proba(test_en)

In [23]:
result

array([[0.95229645, 0.04770355],
       [0.63997861, 0.36002139],
       [0.96753337, 0.03246663],
       ...,
       [0.67951291, 0.32048709],
       [0.98623286, 0.01376714],
       [0.85242809, 0.14757191]])

In [24]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['y'] = result[:,1]

In [26]:
submission

,id,y
0,750000,0.047704
1,750001,0.360021
2,750002,0.032467
3,750003,0.010959
4,750004,0.094860
...,...,...
249995,999995,0.019163
249996,999996,0.346481
249997,999997,0.320487
249998,999998,0.013767


In [25]:
submission.to_csv("submission.csv", index=False)